<a href="https://colab.research.google.com/github/hanansuk/guns_n_roses/blob/main/impute_cdc_monthly_state_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
token = 'GHSAT0AAAAAACDOBHUZHZ23H43256K5C6HSZE3DAMQ'
suppressed = pd.read_csv(f'https://raw.githubusercontent.com/hanansuk/guns_n_roses/main/cdc_by_state_month_with_suppressed_values.csv?token={token}')

In [3]:
token = 'GHSAT0AAAAAACDOBHUYIOZPMFDF4ILETM7MZE3DBOA'
aggregate = pd.read_csv(f'https://raw.githubusercontent.com/hanansuk/guns_n_roses/main/cdc_by_state.csv?token={token}')

In [4]:
suppressed.head()

,State,State.Code,Year,Year.Code,Month,Month.Code,Deaths
0,Alabama,1,2021,2021,"Jan., 2021",2021/01,122
1,Alabama,1,2021,2021,"Feb., 2021",2021/02,104
2,Alabama,1,2021,2021,"Mar., 2021",2021/03,103
3,Alabama,1,2021,2021,"Apr., 2021",2021/04,102
4,Alabama,1,2021,2021,"May, 2021",2021/05,108


In [5]:
aggregate.head()

,State,State.Code,Year,Year.Code,Deaths,Population,Crude.Rate,YoY
0,Alabama,1,1999,1999,787,4430141,17.8,NaN
1,Alabama,1,2000,2000,764,4447100,17.2,-0.030105
2,Alabama,1,2001,2001,737,4467634,16.5,-0.036635
3,Alabama,1,2002,2002,724,4480089,16.2,-0.017956
4,Alabama,1,2003,2003,764,4503491,17.0,0.052356


In [14]:
combined = suppressed.merge(aggregate, on=['State.Code', 'Year'])
combined.rename({
    'State_x': 'state',
    'Year': 'year',
    'Month.Code': 'period',
    'Deaths_x': 'monthly_gun_deaths',
    'Deaths_y': 'yearly_gun_deaths'
}, axis=1, inplace=True)
combined = combined[['state', 'year', 'period', 'monthly_gun_deaths', 'yearly_gun_deaths']]
combined['monthly_gun_deaths'] = [int(x) if x != 'Suppressed' else x for x in combined['monthly_gun_deaths']]
combined['yearly_gun_deaths'] = combined['yearly_gun_deaths'].astype('int')

In [15]:
missing_values_per_year = (
    combined[combined['monthly_gun_deaths'] == 'Suppressed']
    .groupby(['state', 'year'])
    .count()
    .period
    .reset_index())
combined = combined.merge(missing_values_per_year, on=['state', 'year'], how='left')
combined.rename({'period_y': 'missing_values_per_year', 'period_x': 'period'}, axis=1, inplace=True)

In [17]:
combined

,state,year,period,monthly_gun_deaths,yearly_gun_deaths,missing_values_per_year
0,Alabama,2021,2021/01,122,1304,NaN
1,Alabama,2021,2021/02,104,1304,NaN
2,Alabama,2021,2021/03,103,1304,NaN
3,Alabama,2021,2021/04,102,1304,NaN
4,Alabama,2021,2021/05,108,1304,NaN
...,...,...,...,...,...,...
14071,Wyoming,2020,2020/08,17,151,5.0
14072,Wyoming,2020,2020/09,13,151,5.0
14073,Wyoming,2020,2020/10,Suppressed,151,5.0
14074,Wyoming,2020,2020/11,Suppressed,151,5.0


In [18]:
total_monthly_not_missing = (
    combined[combined['monthly_gun_deaths'] != 'Suppressed']
    .groupby(['state', 'year'])
    .sum(numeric_only=False)
    .monthly_gun_deaths
    .reset_index())
combined = combined.merge(total_monthly_not_missing, on=['state', 'year'], how='left')
combined.rename({'monthly_gun_deaths_y': 'total_monthly_not_missing', 'monthly_gun_deaths_x': 'monthly_gun_deaths'}, axis=1, inplace=True)

In [20]:
combined['imputed_monthly'] = (combined.yearly_gun_deaths - combined.total_monthly_not_missing) / combined.missing_values_per_year
combined['monthly_gun_deaths'] = [x.imputed_monthly if x.monthly_gun_deaths == 'Suppressed' else x.monthly_gun_deaths for _, x in combined.iterrows()]

In [22]:
res = combined[['state', 'year', 'period', 'monthly_gun_deaths']]

In [25]:
res['period'] = pd.to_datetime(res.period, format='%Y/%m')

<ipython-input-25-3faaa78f7384>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['period'] = pd.to_datetime(res.period, format='%Y/%m')


In [27]:
from google.colab import files
res.to_csv('cdc_monthly_state_gun_deaths_imputed.csv', encoding = 'utf-8-sig')
files.download('cdc_monthly_state_gun_deaths_imputed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>